In [319]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense, Convolution2D, ELU, Dropout
from keras.models import Model, Sequential
from keras import backend
from keras import callbacks
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import cv2
import matplotlib.image as mpimg
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import json
%matplotlib inline

In [290]:
tf.python.control_flow_ops = tf
backend.image_dim_ordering()

'tf'

In [320]:
def save_model(model, model_name):
    model.save_weights(model_name+"weights.keras", True)
    with open(model_name+'model.json', 'w') as outfile:
        json.dump(model.to_json(), outfile)

In [3]:
def make_model_vgg16():
    # vgg16
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    #Create input format)
    input = Input(name = 'image_input, shape=(224,224,3))

    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(input)

    # freeze vgg16 conv layers
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    #model_vgg16_conv.summary()

    # Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dense(1, activation='linear', name='predicton_steering')(x)

    # Create model 
    return Model(input=input, output=x)

In [262]:
def make_model_nvidia():
    # based on this paper:
    # http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
    input = Input(shape=(80,160,3),name = 'image_input')

    x=Convolution2D(24, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu", name='conv1')(input)
    x=Convolution2D(36, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu", name='conv2')(x)
    x=Convolution2D(48, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu", name='conv3')(x)
    x=Convolution2D(64, 3, 3, border_mode="valid", subsample=(1, 1), activation="relu", name='conv4')(x)
    x=Convolution2D(64, 3, 3, border_mode="valid", subsample=(1, 1), activation="relu", name='conv5')(x)
    x=Flatten(name='flatten')(x) 
    x=Dense(1164, activation="relu", name='dense1')(x)
    x=Dense(100, activation="relu", name='dense2')(x)
    x=Dense(50, activation="relu", name='dense3')(x)
    x=Dense(10, activation="relu", name='dense4')(x)
    x=Dense(1, activation="linear", name='dense5')(x)
    
    # Create model
    return Model(input=input, output=x)

In [335]:
def make_model_commaai():
    model = Sequential()
    #model.add(Lambda(lambda x: x/127.5 - 1.,
    #        input_shape=(ch, row, col),
    #        output_shape=(ch, row, col)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same", input_shape = (80,160,3)))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    #model.compile(optimizer="adam", loss="mse")
    return model


In [336]:
my_model=make_model_commaai()
my_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_11 (Convolution2D) (None, 20, 40, 16)    3088        convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
elu_11 (ELU)                     (None, 20, 40, 16)    0           convolution2d_11[0][0]           
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 10, 20, 32)    12832       elu_11[0][0]                     
____________________________________________________________________________________________________
elu_12 (ELU)                     (None, 10, 20, 32)    0           convolution2d_12[0][0]           
___________________________________________________________________________________________

In [324]:
base_path = os.getcwd()

In [325]:
def preprocess_data_log(base_path, csv_file_name, split=0.2):
    # read log file
    log = pd.read_csv(base_path + '/' + csv_file_name)
    # modify img file name column to contain the full path
    log['center'] = base_path + '/'+ log['center']
    # extract img file names
    img_names = log['center'].tolist()
    # extract angles
    angles = log['steering']
    
    img_names_train, img_names_test, angles_train, angles_test = train_test_split(img_names, angles, test_size=split, random_state=42)
    
    return ( list(zip(img_names_train, angles_train)), list(zip(img_names_test, angles_test)) )

In [326]:
(train_data_list, test_data_list) = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv')
#train_data_list[0:3]

In [337]:
my_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
#my_model.compile(optimizer='adam', loss='mean_absolute_percentage_error', metrics=['accuracy'])
#my_model.compile(optimizer='adam', loss='mse')

In [338]:
def batch_gen(name, data_list, out_img_size, batch_size):
    # data_list is in this form: ['file_name', angle]
    # out_img_size is a shape of the output image: (x, y, color)
    
    # create batch_size np arrays as placeholders for imgages and angles
    X_train = np.empty((batch_size,) +  out_img_size, dtype = np.float32)
    y_train = np.empty(batch_size, dtype = np.float32)
    
    # col & rows are reversed for cv2.resize
    resize_shape = (out_img_size[1], out_img_size[0])
    
    while True:
        # shuffle data
        shuffle(data_list)
        for offset in range(0, len(data_list), batch_size):
            end = offset + batch_size
            batch_subset = data_list[offset:end]
            for i, _ in enumerate(batch_subset):
                #load image
                X_train[i] = (cv2.resize(cv2.imread(batch_subset[i][0]), resize_shape).astype(np.float32))/128. - 1.
                #X_train[i] = np.expand_dims(im, axis=0)
                # load angle
                y_train[i] = batch_subset[i][1]
                
            batch_X, batch_y = X_train[:len(batch_subset)], y_train[:len(batch_subset)]
            #print(name)
            yield (batch_X, batch_y)

In [329]:
#g=batch_gen(out[0:6], (160, 320, 3), 2)

In [330]:
#(im, a) = next(g)
len(train_data_list)

6428

In [331]:
len(test_data_list)

1608

In [277]:
#plt.imshow(cv2.cvtColor((im[0]*255.+128.).astype(np.uint8), cv2.COLOR_BGR2RGB))
#im0 = cv2.resize(cv2.imread(train_data_list[0][0]), (224, 224))
#plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))

In [278]:
class printbatch(callbacks.Callback):
    def on_batch_end(self, epoch, logs={}):
        print(logs)
        
pb = printbatch()

In [339]:
train_gen = batch_gen("train_gen", train_data_list, (80, 160, 3), 64)
test_gen = batch_gen("test_gen", test_data_list, (80, 160, 3), 64)

In [ ]:
history = my_model.fit_generator(
    generator=train_gen, 
    validation_data=test_gen, 
    nb_val_samples=len(test_data_list), 
    samples_per_epoch=len(train_data_list), 
    nb_epoch=200,
    max_q_size=1)

Epoch 1/200
6428/6428 [==============================] - 15s - loss: 0.0166 - acc: 0.5352 - val_loss: 0.0116 - val_acc: 0.5709
Epoch 2/200
6428/6428 [==============================] - 14s - loss: 0.0112 - acc: 0.5356 - val_loss: 0.0107 - val_acc: 0.5709
Epoch 3/200
6428/6428 [==============================] - 14s - loss: 0.0097 - acc: 0.5356 - val_loss: 0.0108 - val_acc: 0.5709
Epoch 4/200
6428/6428 [==============================] - 14s - loss: 0.0084 - acc: 0.5356 - val_loss: 0.0118 - val_acc: 0.5709
Epoch 5/200
6428/6428 [==============================] - 14s - loss: 0.0079 - acc: 0.5356 - val_loss: 0.0113 - val_acc: 0.5709
Epoch 6/200
6428/6428 [==============================] - 14s - loss: 0.0075 - acc: 0.5356 - val_loss: 0.0106 - val_acc: 0.5709
Epoch 7/200
6428/6428 [==============================] - 14s - loss: 0.0071 - acc: 0.5356 - val_loss: 0.0113 - val_acc: 0.5709
Epoch 8/200
6428/6428 [==============================] - 14s - loss: 0.0067 - acc: 0.5359 - val_loss: 0.0110 - 

In [321]:
save_model(my_model, 'first_try')

In [236]:
train_gen = batch_gen("train_gen", train_data_list, (80, 160, 3), 1)

In [237]:
im, t = next(train_gen)

In [250]:
#im[0]

In [196]:
im0 = (im[0]*128.+128.).astype(np.uint8)

In [ ]:
train_gen = batch_gen("train_gen", train_data_list, (80, 160, 3), 25)

In [312]:
im, s = next(train_gen)

In [313]:
my_model.predict(im)

array([[ 0.02251387],
       [-0.01097528],
       [-0.07046968],
       [ 0.01269822],
       [-0.02309453],
       [ 0.06392274],
       [ 0.11117297],
       [ 0.07657015],
       [ 0.11020656],
       [ 0.04373782],
       [-0.08349971],
       [ 0.04887722],
       [ 0.03102045],
       [ 0.10972355],
       [ 0.07347102],
       [-0.01967359],
       [-0.05685198],
       [ 0.00174202],
       [ 0.04377614],
       [-0.16624372],
       [ 0.08014531],
       [ 0.03017185],
       [-0.04866779],
       [ 0.05054331],
       [-0.00949066]], dtype=float32)

In [314]:
s

array([ 0.        ,  0.        ,  0.        ,  0.        , -0.135712  ,
        0.        ,  0.100034  ,  0.1287396 ,  0.1287396 ,  0.01391724,
        0.        ,  0.        ,  0.        ,  0.0617599 ,  0.1287396 ,
        0.        ,  0.        ,  0.        ,  0.        , -0.06925154,
        0.        , -0.06925154,  0.        ,  0.        ,  0.01391724], dtype=float32)